In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import torchvision

#from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical

from tqdm import tqdm

import json
import nltk
from nltk import word_tokenize

from torch.utils.data.dataloader import default_collate


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:

idfilename = 'training_data/id.txt'
datadirname = 'training_data/feat/'
labelfilename = 'training_label.json'

with open("DIC_word_index.json") as f:
    DIC_word_index = json.load(f)

with open("DIC_index_word.json") as f:
    DIC_index_word = json.load(f)

In [4]:


class SeqAttNet(torch.nn.Module):
    """

    Args:

    """

    def __init__(self, latent_dim, VOCAB_SZ):
        super(SeqAttNet, self).__init__()

        self.hidden_dim = latent_dim

        self.lstm1 = torch.nn.GRU(4096, latent_dim,  num_layers=1, batch_first = True)
        self.lstm2 = torch.nn.GRU(VOCAB_SZ, latent_dim,  num_layers=1, batch_first = True)
        self.soft = torch.nn.Softmax(-2)


        

        self.trainable_W = torch.nn.Sequential(
            torch.nn.Linear(latent_dim ,  latent_dim),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.2),
        )
        
        self.last_layer = torch.nn.Sequential(
            torch.nn.Linear( 2 * latent_dim,  VOCAB_SZ),
            torch.nn.Tanh(),
            torch.nn.Softmax(-1)
        )
    
    
    def encoder_lstm(self, x):
        #input of shape (seq_len, batch, input_size):
        #print(x.shape)
        x, hiddens = self.lstm1(x)
        
        #x shape -> [seq_n, batch_size, latent_dim * 2]
        return x, hiddens
    
    def decoder_lstm(self, x, hiddens):
        #input of shape ( batch,seq_len, input_size):

        x, hiddens = self.lstm2(x, hiddens)

        #x shape -> [batch_size,seq_n,  latent_dim * 2]
        return x, hiddens
    

    def forward(self, video_feature, sent):

        #print(sent.shape) torch.Size([2, 15, 6087])

        e_output, hiddens = self.encoder_lstm(video_feature)
        uW = self.trainable_W(e_output)
        
        first = 1       
        
        for word in sent.transpose(1,0):
            #print(word.shape) torch.Size([2, 6087])
            word = torch.unsqueeze(word , 1)

            #print(word.shape)
            #one for each word, therefore d_output = d_state
            d_output, hiddens = self.decoder_lstm(word, hiddens)
            uWv = torch.matmul(uW, d_output.transpose(2,1))
            #print(uWv.shape) torch.Size([2, 80, 1])
            alpha = self.soft(uWv)
            #print(uWv)
            #print(alpha)
            #input("")
            new_context = e_output.transpose(2,1) @ alpha;
            #print(e_output.shape) torch.Size([2, 80, 256])
            #print(alpha.shape) torch.Size([2, 80, 1])
            
            #print(d_output.shape) torch.Size([2, 1, 256])
            #print(new_context.shape) torch.Size([2, 256, 1])          
            pre = torch.cat((d_output, new_context.transpose(2,1)), 2)
            if(first):
                first = 0;
                last_layer_input = pre
            else:
                last_layer_input = torch.cat((last_layer_input, pre), 1)
            #print(last_layer_input.shape) torch.Size([2, 1, 512])
            
        logits = self.last_layer(last_layer_input)

        #logits = torch.stack(logits, 1)
        return logits
            


In [5]:
def to_words(ys):       
    tokens = []
    for catagorical_word in ys:
        index = np.argmax(catagorical_word.cpu())
        #print(index)
        if str(index.cpu().item()) in DIC_index_word:
            tokens.append(DIC_index_word[str(index.cpu().item())])
        else:
            tokens.append('<unk>');
    return tokens

In [6]:
def _run_iter(batch, training, model, loss_function):
    with torch.no_grad():
        image_feature = batch['encoder_x'].float().to(device)
        sent = batch['decoder_x'].float().to(device)
    output = model.forward(image_feature, sent)
    #print(output.shape)
    #print(batch['decoder_y'].shape)
    #input("")
    #print(output)
    #input("")
    loss = loss_function(output, batch['decoder_y'].float().to(device))
    return output, loss

In [7]:
def _run_epoch(dataloader, training, model, optimizer, loss_function):
    model.train(training)
    if training:
        iter_in_epoch = min(len(dataloader), 1000000)
        description = 'train'
    else:
        iter_in_epoch = len(dataloader)
        description = 'test'
    grad_accumulate_steps = 1
    trange = tqdm(enumerate(dataloader), total=iter_in_epoch, desc=description)
    loss = 0
    for i, batch in trange:   
        if training and i >= iter_in_epoch:
            break

        if training:
            #print("batch:{}".format(batch))
            #print(batch['context'].dtype)
            optimizer.zero_grad()
            output, batch_loss = _run_iter(batch, training, model, loss_function)            
            
            batch_loss /= grad_accumulate_steps
            
            if i % grad_accumulate_steps == 0:
                optimizer.zero_grad()
            
            batch_loss.backward()

            if (i + 1) % grad_accumulate_steps == 0:
                optimizer.step()
            if(i == 0):
                print(to_words(batch['decoder_x'][0]))
                print(to_words(output[0]))
        else:
            with torch.no_grad():
                output, batch_loss = _run_iter(batch, training, model, loss_function)
                if(i == 0):
                    print(to_words(batch['decoder_x'][0]))
                    print(to_words(output[0]))
                
        loss += batch_loss.item()

    loss /= iter_in_epoch
    print('loss=%f\n' % loss)

In [8]:
def Sent2Seq(sent):
    #print(sent)    
    tokens = word_tokenize(sent.lower())
    ret = []
    for word in tokens:
        if(word in DIC_word_index):
            ret.append(DIC_word_index[word])
        else:
            ret.append(DIC_word_index["<unk>"])
    return ret



In [9]:
def pad_post_zero(a, length):
    ret = []
    for _list in a:
        if(len(_list) < length):
            for ct in range(len(_list),length,1):
                _list.append(0)
        if(len(_list) > length):
            _list = _list[:length]
            
        ret.append(_list)
    return ret

In [10]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
        dtype: The data type expected by the input, as a string
            (`float32`, `float64`, `int32`...)
    # Returns
        A binary matrix representation of the input. The classes axis
        is placed last.
    # Example
    ```python
    # Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
    > labels
    array([0, 2, 1, 2, 0])
    # `to_categorical` converts this into a matrix with as many
    # columns as there are classes. The number of rows
    # stays the same.
    > to_categorical(labels)
    array([[ 1.,  0.,  0.],
           [ 0.,  0.,  1.],
           [ 0.,  1.,  0.],
           [ 0.,  0.,  1.],
           [ 1.,  0.,  0.]], dtype=float32)
    ```
    """

    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical


In [11]:
def my_collate(datas):
    batch = {}
    # collate lists
    batch['decoder_x'] = torch.tensor([data['decoder_x'] for data in datas])
    batch['decoder_y'] = torch.tensor([data['decoder_y'] for data in datas])
    batch['encoder_x'] = torch.tensor([data['encoder_x'] for data in datas])
    return batch

In [12]:
# loading training data
encode_x = []
video_id = {}
for i,video_name in enumerate(open(idfilename)):
    video_name = video_name[:-1]
    x = np.load(datadirname + video_name + ".npy")
    encode_x.append(x)
    video_id[video_name] = i

encode_x = np.array(encode_x)

TRAIN_SZ = len(encode_x)
decode_x = [[]]*TRAIN_SZ
decode_y = [[]]*TRAIN_SZ

In [13]:
VOCAB_SZ = len(DIC_word_index)
MAX_SEQ_LEN = 0;
# loading decoder data
rawlabels = json.load(open(labelfilename, 'r'))
for data in rawlabels:    
    index = video_id[data['id']]
    #print(index)
    sent =  data['caption'][-1] # select one sentence for now
    # TODO: implement Sent2Seq
    decode_x[index] = [1] + Sent2Seq(sent)
    decode_y[index] = Sent2Seq(sent) + [2]
    if(len(decode_x[index]) > MAX_SEQ_LEN):
        MAX_SEQ_LEN = len(decode_x[index])

In [ ]:
###valid#####
valid_encode_x = []
valid_video_id = {}
for i,video_name in enumerate(open('testing_data/id.txt')):
    video_name = video_name[:-1]
    x = np.load("testing_data/feat/" + video_name + ".npy")
    valid_encode_x.append(x)
    valid_video_id[video_name] = i

valid_encode_x = np.array(valid_encode_x)

VALID_SZ = len(valid_encode_x)
valid_decode_x = [[]]*VALID_SZ
valid_decode_y = [[]]*VALID_SZ

In [ ]:
rawlabels = json.load(open('testing_label.json', 'r'))
for data in rawlabels:    
    index = valid_video_id[data['id']]
    #print(index)
    sent =  data['caption'][-1] # select one sentence for now
    # TODO: implement Sent2Seq
    valid_decode_x[index] = [1] + Sent2Seq(sent)
    valid_decode_y[index] = Sent2Seq(sent) + [2]


In [ ]:
lr = 1e-3  # learning rate
MAX_SEQ_LEN = 10
batch_size = 2
latent_dim = 256
iter_in_epoch = 500

In [ ]:
decode_x = pad_post_zero(decode_x, MAX_SEQ_LEN)
decode_y = pad_post_zero(decode_y, MAX_SEQ_LEN)
#decode_x = pad_sequences(decode_x, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')
#decode_y = pad_sequences(decode_y, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')

decode_y = to_categorical(decode_y, VOCAB_SZ)
decode_x = to_categorical(decode_x, VOCAB_SZ)

datas = []
for a,b,c in zip(decode_x, decode_y , encode_x):
    data = {}

    data['decoder_x'] = a
    data['decoder_y'] = b;
    data['encoder_x'] = c;
    datas.append(data)


In [ ]:
valid_decode_x = pad_post_zero(valid_decode_x, MAX_SEQ_LEN)
valid_decode_y = pad_post_zero(valid_decode_y, MAX_SEQ_LEN)
#decode_x = pad_sequences(decode_x, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')
#decode_y = pad_sequences(decode_y, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')

valid_decode_y = to_categorical(valid_decode_y, VOCAB_SZ)
valid_decode_x = to_categorical(valid_decode_x, VOCAB_SZ)

valid_datas = []
for a,b,c in zip(valid_decode_x, valid_decode_y , valid_encode_x):
    valid_data = {}

    valid_data['decoder_x'] = a
    valid_data['decoder_y'] = b;
    valid_data['encoder_x'] = c;
    valid_datas.append(valid_data)

In [ ]:
print(len(decode_x))
print(len(decode_y))
print(len(encode_x))
#for i in enumerate(datas):
#    print(i)

1450
1450
1450


In [ ]:
model = SeqAttNet(latent_dim, VOCAB_SZ)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_function = torch.nn.BCELoss()

In [ ]:
epoch = 0;
max_epochs = 100  # how many epochs to train for
while epoch < max_epochs:
    # train and evaluate train score
    print('training %i' % epoch)

    # train epoch
    dataloader = torch.utils.data.DataLoader(datas, batch_size = batch_size, collate_fn = my_collate)
    log_train = _run_epoch(dataloader, True, model, optimizer, loss_function)

    # test epoch
    print('evaluating %i' % epoch)
    dataloader = torch.utils.data.DataLoader(valid_datas, batch_size = batch_size, collate_fn=my_collate)
    log_valid = _run_epoch(dataloader, False, model, optimizer, loss_function)

    epoch += 1





train:   0%|          | 2/725 [00:00<00:59, 12.12it/s]

training 0
['<bos>', 'the', 'horse', 'pooped', 'on', 'the', 'lady', 'who', 'was', 'underneath']
['chees', 'gives', 'gives', 'escaped', 'escaped', 'escaped', 'escaped', 'gives', 'otherwise', 'gives']


test:   4%|▍         | 2/50 [00:00<00:03, 13.84it/s]

loss=0.001367

evaluating 0
['<bos>', 'the', 'man', 'is', 'seasoning', 'the', 'chicken', '.', '<pad>', '<pad>']
['young', 'young', 'playing', 'playing', 'tail', 'tail', 'tail', 'half', 'pot', 'pot']


train:   0%|          | 2/725 [00:00<00:58, 12.36it/s]

...

test:   4%|▍         | 2/50 [00:00<00:03, 13.81it/s]

loss=0.001270

evaluating 83
['<bos>', 'the', 'man', 'is', 'seasoning', 'the', 'chicken', '.', '<pad>', '<pad>']
['someone', 'man', 'put', 'the', 'the', 'the', 'with', 'the', 'the', 'the']


train:   0%|          | 2/725 [00:00<00:57, 12.54it/s]

loss=0.001366

training 84
['<bos>', 'the', 'horse', 'pooped', 'on', 'the', 'lady', 'who', 'was', 'underneath']
['two', 'man', 'is', 'off', 'his', 'man', 'on', 'a', 'a', 'the']


train:  47%|████▋     | 338/725 [00:26<00:30, 12.58it/s]

In [ ]:
torch.save({'model': model.state_dict(),'optimizer': optimizer.state_dict()}, "Attention_model.h5")

In [ ]:
idfilename_t = 'testing_data/id.txt'
datadirname_t = 'testing_data/feat/'

In [ ]:
#====================TESTING===================#
BOS = "<bos>"
EOS = "<eos>"
PAD = "<pad>"
UNK = "<unk>"



TEST_SZ = 0
# loading testing data

encode_x = []
video_id = {}
for i,video_name in enumerate(open(idfilename_t)):
    #lb contains '\n', therefore lb[:-1]
    video_name = video_name[:-1]
    x = np.load(datadirname_t + video_name + ".npy")
    encode_x.append(x)
    video_id[video_name] = i
    TEST_SZ += 1

TEST_SZ = len(encode_x)
decode_x = [[DIC_word_index[BOS]]]*TEST_SZ
decode_x = pad_post_zero(decode_x, MAX_SEQ_LEN)
decode_x = to_categorical(decode_x, VOCAB_SZ)

In [ ]:
datas = []
for a,b in zip(decode_x, encode_x):
    data = {}
    data['decoder_x'] = a
    data['encoder_x'] = b;
    #0 for no reason, need to pass it without error 
    data['decoder_y'] = 0;
    datas.append(data)

In [ ]:
model.eval()
#predict mode
dataloader = torch.utils.data.DataLoader(datas, batch_size = batch_size, collate_fn=my_collate)
predict_label = []
ys_ =[]
with torch.no_grad():
    for batch in tqdm(dataloader):
        image_feature = batch['encoder_x'].float().to(device)
        sent = batch['decoder_x'].float().to(device)
        batch_y_ = model.forward(image_feature, sent)
        ys_.append(batch_y_)
    ys_ = torch.cat(ys_, 0)


In [ ]:
print(ys_.shape)
#print(DIC_index_word)

In [ ]:
with open('attention_predict_label.txt', 'w') as f:
    for i,video_name in enumerate(open(idfilename_t)):
        video_name = video_name[:-1]
        tokens = to_words(ys_[i])      
        predict = " ".join(tokens)

        if(predict[-1] == "."):
            predict  = predict[:-1]
        #print(OUTPUTS[int(_id)])
        #input("")
        f.write(str(video_name) + "," + predict +"\n")
        print(str(video_name) + "," + predict +"\n")


In [ ]:
import os
os.system("python bleu_eval.py attention_predict_label.txt")